In [3]:
# Import necessary modules
#pip install langdetect
import requests
from bs4 import BeautifulSoup
import numpy as np
import re
import langdetect 
import pandas as pd
import nltk
from langdetect import detect
import matplotlib.pyplot as plt
import pandas as pd

final_data = pd.read_csv(r"C:\Users\lisbe\OneDrive\11. Semester\Social Data Science\Modules\final_data.csv", index_col=0).reset_index()
df = final_data.copy()

#Define todays date and enddate for data
from datetime import date
today = date.today().strftime("%Y%m%d") # to get format 20080101
start = '20100101'


#Make lists of dates, areas, categories and subcategories
#all unique dates
dates = pd.DataFrame(pd.date_range(start=start, end=today, freq='D').strftime("%Y%m%d").tolist())
dates = dates.iloc[pd.to_datetime(dates.index).argsort()] #sort
dates.columns = ['Date']

#all unique month-year
months = pd.DataFrame(pd.date_range(start=start, end=today, freq='MS').strftime("%b-%Y").tolist()) 
months = months.iloc[pd.to_datetime(months.index).argsort()] #sort
months.columns = ['Month-Year']
#monthlist = [str(date).strip("()").strip("datetime.date").replace(',','') for date in monthlist]

#all unique areas
areas = pd.DataFrame(np.unique(final_data['Geographical Area']))
areas.columns = ['Geographical Area']

#all unique categories
categories = pd.DataFrame(np.unique(final_data['Job Category']))
categories.columns = ['Job Category']

#all unique subcategories
subcategories = pd.DataFrame(np.unique(final_data['Job Subcategory']))
subcategories.columns = ['Job Subcategory']

'''#Figure 1a shows the actual number of remote job postings on Jobindex at a given point in time in the period 2015-2020. 
The bars are further split into job categories (industries).'''

#Count the number of remote job postings per month-year
total_remote_date = pd.DataFrame(final_data.groupby('Month-Year')['Remote'].sum()).reset_index()
total_remote_date.columns = ['Month-Year','Remote Postings Published']

In [4]:
# We want a time series with two columns : 'ds' and 'y' where ds is the date

total_remote_date['Month-Year']= pd.to_datetime(total_remote_date['Month-Year'])
total_remote_date.columns = ['ds','y']
total_remote_date.sort_values(by=['ds'], ascending=True).reset_index(drop=True)

ts = total_remote_date.sort_values(by=['ds'], ascending=True).reset_index(drop=True)[:121]

In [ ]:
from fbprophet import Prophet
from fbprophet.diagnostics

def mean_absolute_

In [ ]:
# Fitting the model to our data set
model = Prophet()
model.fit(ts)

# Creating an empty data set for forecasting

from pandas import DataFrame
future= pd.date_range('2016-01-01','2017-01-01', freq='1M')-pd.offsets.MonthBegin(1)
future = future.tolist()

future = DataFrame(future,columns=['ds'])
future['ds'] = pd.to_datetime(future['ds'])
predict_2016 = model.predict(future)

# Making forecast for february to july 2020

forecast= pd.date_range('2010-01-01','2020-08-01', freq='1M')-pd.offsets.MonthBegin(1)
forecast = forecast.tolist()

forecast = DataFrame(forecast,columns=['ds'])
forecast['ds'] = pd.to_datetime(forecast['ds'])
forecast = model.predict(forecast)

true_data = total_remote_date.sort_values(by=['ds'], ascending=True).reset_index(drop=True)[:127]
forecast = forecast[['ds','yhat','yhat_lower','yhat_upper']]

# create figure and axis objects with subplots()

plot = pd.merge(true_data, forecast, on='ds', how='right')
ticks= pd.date_range('2010-12-01','2021-08-01', freq='1Y')-pd.offsets.MonthBegin(12)
fig,ax = plt.subplots(figsize=(11,7))
ax.plot(plot['ds'],plot['yhat'], color="red", linewidth=0.9, label = 'Predicted')
ax.plot(plot['ds'],plot['y'], color="blue", linewidth=0.9, label = 'Actual')
ax.set_xlabel("Month-Year",fontsize=15)
ax.set_ylabel("Monthly Remote Job Listings", fontsize=15)
plt.xticks(ticks,fontsize=12,rotation=20)
plt.yticks(fontsize=12)
ax.legend(fontsize=15)
plt.savefig('forecast.png')